# Title: msticpy - nbwidgets
## Description:
This contains a few aggregated widgets using IPyWidgets that help speed things up during an investigation.



<a id='contents'></a>
## Table of Contents
- [Setting query start/end times](#QueryTime)
- [Simple time range](#Lookback)
- [Selecting and Displaying Alerts](#AlertSelector)
- [Selecting from list or dict](#SelectString)
- [Getting a value from environment](#GetEnvironmentKey)


In [1]:
# Imports
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

from IPython.display import display, Markdown
import pandas as pd
# Import nbtools package
from msticpy.nbtools import *


<a id='QueryTime'></a>[Contents](#contents)
## QueryTime

This widget is used to specify time boundaries - designed to be used with the built-in msticpy queries and custom queries.
The `start` and `end` times are exposed as datetime properties.

```
QueryTime.

Composite widget to capture date and time origin
and set start and end times for queries.

Parameters
----------
QueryParamProvider : QueryParamProvider
    Abstract base class

Parameters
----------
origin_time : datetime, optional
    The origin time (the default is `datetime.utcnow()`)
label : str, optional
    The description to display
    (the default is 'Select time ({units}) to look back')
before : int, optional
    The default number of `units` before the `origin_time`
    (the default is 60)
after : int, optional
    The default number of `units` after the `origin_time`
    (the default is 10)
max_before : int, optional
    The largest value for `before` (the default is 600)
max_after : int, optional
    The largest value for `after` (the default is 100)
units : str, optional
    Time unit (the default is 'min')
    Permissable values are 'day', 'hour', 'minute', 'second'
    These can all be abbreviated down to initial characters
    ('d', 'm', etc.)
auto_display : bool, optional
    Whether to display on instantiation (the default is False)
```

In [2]:
q_times = nbwidgets.QueryTime(units='day', max_before=20, before=5, max_after=1)
q_times.display()

HTML(value='<h4>Set query time boundaries</h4>')

In [3]:
print(q_times.start, '....', q_times.end)

2020-06-07 19:13:56.217508 .... 2020-06-13 19:13:56.217508


Keep multiple query boundaries aligged by having QueryTime instances reference the time of the same alert or event, or have them chained from one another by referencing the origin_time of an earlier QueryTimes object

In [4]:
from datetime import datetime, timedelta
class MyAlert:
    pass
alert = MyAlert()
alert.TimeGenerated = datetime.utcnow() - timedelta(15)
alert.TimeGenerated

q_times1 = nbwidgets.QueryTime(units='hour', max_before=20, before=1, max_after=1, 
                             origin_time=alert.TimeGenerated, auto_display=True)

q_times2 = nbwidgets.QueryTime(units='hour', max_before=20, before=4, max_after=2, 
                             origin_time=alert.TimeGenerated, auto_display=True)

HTML(value='<h4>Set query time boundaries</h4>')

HTML(value='<h4>Set query time boundaries</h4>')

In [5]:
alert.TimeGenerated = datetime.utcnow()
q_times1 = nbwidgets.QueryTime(units='hour', max_before=20, before=1, max_after=1, 
                             origin_time=alert.TimeGenerated, auto_display=True)

q_times2 = nbwidgets.QueryTime(units='hour', max_before=20, before=4, max_after=2, 
                             origin_time=q_times2.origin_time, auto_display=True)

HTML(value='<h4>Set query time boundaries</h4>')

HTML(value='<h4>Set query time boundaries</h4>')

In [6]:
# Use the values in a query
my_kql = f'''
SecurityAlert 
| where TimeGenerated >= datetime({q_times1.start})
| where TimeGenerated <= datetime({q_times1.end})'''
print(my_kql)


SecurityAlert 
| where TimeGenerated >= datetime(2020-06-12 18:13:56.437039)
| where TimeGenerated <= datetime(2020-06-12 20:13:56.437039)


<a id='Lookback'></a>[Contents](#contents)
## Lookback
Simpler version with single slider value

Docstring:
`nbtools.Lookback?`

In [7]:
alert.TimeGenerated = datetime.utcnow() - timedelta(5)
lb = nbwidgets.Lookback(origin_time=alert.TimeGenerated, auto_display=True, max_value=48)

IntSlider(value=4, description='Select time (hour) to look back', layout=Layout(height='50px', width='60%'), m…

In [8]:
print(lb.start, '....', lb.end)

2020-06-07 15:13:56.578558 .... 2020-06-07 19:13:56.578558


<a id='AlertSelector'></a>[Contents](#contents)
## Alert Browser

```
SelectAlert.

View list of alerts and select one for investigation.
Optionally provide and action to call with the selected alert as a parameter
(typically used to display the alert.)

Attributes:
    selected_alert: the selected alert
    alert_id: the ID of the selected alert
    alerts: the current alert list (DataFrame)
Init docstring:
Create a new instance of AlertSelector.

Parameters
----------
alerts : pd.DataFrame
    DataFrame of alerts.
action : Callable[..., None], optional
    Optional function to execute for each selected alert.
    (the default is None)
columns : list, optional
    Override the default column names to use from `alerts`
    (the default is ['StartTimeUtc', 'AlertName',
    'CompromisedEntity', 'SystemAlertId'])
auto_display : bool, optional
    Whether to display on instantiation (the default is False)
```

### Simple alert selector
Selected alert is available as `select_alert_widget.selected_alert` property

In [21]:
# Load test data
alerts = pd.read_csv('data/alertlist.csv')

display(Markdown('### Simple alert selector'))
display(Markdown('Selected alert is available as `select_alert_widget.selected_alert`'))
alert_select = nbwidgets.SelectAlert(alerts=alerts)
alert_select.display()

### Simple alert selector

Selected alert is available as `select_alert_widget.selected_alert`

### Alert selector with action=SecurityAlert'
You can pass a function that returns one or more displayable objects.
You can also pass a class (in this case we're passing `SecurityAlert`) that produces an IPython displayable object.

The `action` class/function is passed the raw alert row as a parameter, as it is selected from the list

In [23]:
alert_select = nbwidgets.SelectAlert(alerts=alerts, action=SecurityAlert)
alert_select.display()

,0
Unnamed: 0,0
TenantId,802d39e1-9d70-404d-832c-2de5e2478eda
StartTimeUtc,2019-01-11 06:31:40
EndTimeUtc,2019-01-12 06:31:40
ProviderAlertId,e0c9484b-ad5f-4161-b73b-388676c05818
SystemAlertId,047f47d6-79b7-4502-824b-97abc4905a73
ProviderName,CustomAlertRule
VendorName,Alert Rule
AlertType,DC local group addition - Demo
AlertName,DC local group addition - Demo


In [26]:
from IPython.display import HTML
security_alert = None

# create a function to get the displayable object
def alert_with_entities(alert):
    return HTML(SecurityAlert(alert).to_html(show_entities=True))
                
alert_select = nbwidgets.SelectAlert(alerts=alerts.query('CompromisedEntity == "MSTICALERTSWIN1"'), 
                                     action=alert_with_entities)
display(Markdown('### Or a more detailed display with extracted entities'))
alert_select

### Or a more detailed display with extracted entities

,77
Unnamed: 0,77
TenantId,802d39e1-9d70-404d-832c-2de5e2478eda
StartTimeUtc,2019-01-15 05:15:11
EndTimeUtc,2019-01-15 05:15:11
ProviderAlertId,4fa0022e-f431-4ba1-ae5f-ee23247e0853
SystemAlertId,2518547714887413669_4fa0022e-f431-4ba1-ae5f-ee23247e0853
ProviderName,Detection
VendorName,Microsoft
AlertType,Digital currency mining related behavior detected
AlertName,Digital currency mining related behavior detected


<a id='SelectItem'></a>[Contents](#contents)
## SelectItem

Similar to AlertSelector but simpler and allows you to use any list or dictionary of items.

```
Selection list from list or dict.

Attributes:
    value : The selected value.
Init docstring:
Select an item from a list or dict.

Parameters
----------
description : str, optional
    The widget label to display (the default is None)
item_list : List[str], optional
    A `list` of items to select from (the default is None)
item_dict : Mapping[str, str], optional
    A `dict` of items to select from. When using `item_dict`
    the keys are displayed as the selectable items and value
    corresponding to the selected key is set as the `value`
    property.
    (the default is None)
action : Callable[..., None], optional
    function to call when item selected (passed a single
    parameter - the value of the currently selected item)
    (the default is None)
auto_display : bool, optional
    Whether to display on instantiation (the default is False)
height : str, optional
    Selection list height (the default is '100px')
width : str, optional
    Selection list width (the default is '50%')
```

In [32]:
# extract the entities from the previously selected alert
security_alert = SecurityAlert(alert_select.selected_alert)
if security_alert is None:
    security_alert = SecurityAlert(alerts.iloc[1])
ent_dict = {ent['Type']:ent for ent in security_alert.entities}

# from IPython.display import HTML

# # create a display function for the entities
# def entity_to_html(entity):
#     e_text = str(entity)
#     e_type = entity.Type
#     e_text = e_text.replace("\n", "<br>").replace(" ", "&nbsp;")
#     return HTML(f"<h3>{e_type}</h3>{e_text}")
    
nbwidgets.SelectItem(item_dict=ent_dict,
                       description='Select an item',
                       action=lambda x: x,
                       auto_display=True);


HostLogonSession(Type=host-logon-session, Account={ 'AdditionalData': {},
  'Host': { 'Additional...)

<a id='GetEnvironmentKey'></a>[Contents](#contents)
## GetEnvironmentKey
Get editable value of environment variable. Common use would be retrieving an API key from your environment or allowing you to paste in a value if the environment key isn't set.

Note setting the variable only persists in the python kernel process running at the time. So you can retrieve it later in the notebook but not in other processes.

In [14]:
nbwidgets.GetEnvironmentKey(env_var='userprofile', auto_display=True);

<a id='SelectSubset'></a>[Contents](#contents)
## SelectSubset
Allows you to select one or multiple items from a list to populate an output set.

```
Class to select a subset from an input list.

    Attributes
    ----------
    selected_values : List[Any]
        The selected item values.
    selected_items : List[Any]
        The selected items label and value
        
Init docstring:
Create instance of SelectSubset widget.

Parameters
----------
source_items : Union[Dict[str, str], List[Any]]
    List of source items - either a dictionary(label, value),
    a simple list or
    a list of (label, value) tuples.
default_selected : Union[Dict[str, str], List[Any]]
    Populate the selected list with values - either
    a dictionary(label, value),
    a simple list or
    a list of (label, value) tuples.
```

In [15]:
# Simple list
items = list(alerts["AlertName"].values)
sel_sub = nbwidgets.SelectSubset(source_items=items)

In [16]:
# Label/Value pair items with a a subset of pre-selected items
items = {v: k for k, v in alerts["AlertName"].to_dict().items()}
pre_selected = {v: k for k, v in alerts["AlertName"].to_dict().items() if "commandline" in v}
sel_sub = nbwidgets.SelectSubset(source_items=items, default_selected=pre_selected)


In [17]:
print("Values:", sel_sub.selected_values, "\n")
print("Items:", sel_sub.selected_items)

Values: [79, 109, 83] 

Items: [('Detected suspicious commandline arguments', 79), ('Detected suspicious commandline used to start all executables in a directory', 109), ('Detected suspicious credentials in commandline', 83)]


## Progress Indicator

In [18]:
from time import sleep
progress = nbwidgets.Progress(completed_len=2000)
for i in range(0, 2100, 100):
    progress.update_progress(new_total=i)
    sleep(0.1)
    
inc_progress = nbwidgets.Progress(completed_len=10)
for i in range(0, 11):
    inc_progress.update_progress(delta=1)
    sleep(0.1)
print("Volume goes to eleven!")

Volume goes to eleven!


## Logon Display
Display logon details for a Windows or Linux logon

In [34]:
win_logons = pd.read_csv("data/host_logons.csv")
user_dict = win_logons.apply(lambda x: f"{x.TargetDomainName}/{x.TargetUserName}   ({x.TimeGenerated})", axis=1).to_dict()
user_dict = {v: k for k, v in user_dict.items()}

from msticpy.nbtools.nbdisplay import format_logon
# create a display function for the entities
def disp_logon(index):
    print
    logons = win_logons[win_logons.index == index]
    return format_logon(logons)
    
acct_select = nbwidgets.SelectItem(item_dict=user_dict,
                       description='Select an item',
                       action=disp_logon,
                       auto_display=True);

Account: MSTICAdminAccount Domain: MSTICAlertsWin1Logon Time: 2019-01-15 04:28:33.090Logon type: 4(Batch)User Id/SID: S-1-5-21-996632719-2361334927-4038480536-500 SID S-1-5-21-996632719-2361334927-4038480536-500 is administrator SID S-1-5-21-996632719-2361334927-4038480536-500 is local machine or domain accountSubject (source) account: WORKGROUP/MSTICAlertsWin1$Logon process: Advapi Authentication: NegotiateSource IpAddress: -Source Host: MSTICAlertsWin1Logon status: nan


#### Display a list of logons

In [36]:
# display a list of logons
display(format_logon(win_logons.head(5)))

Account: MSTICAdminAccount Domain: MSTICAlertsWin1Logon Time: 2019-01-15 05:15:02.980Logon type: 4(Batch)User Id/SID: S-1-5-21-996632719-2361334927-4038480536-500 SID S-1-5-21-996632719-2361334927-4038480536-500 is administrator SID S-1-5-21-996632719-2361334927-4038480536-500 is local machine or domain accountSubject (source) account: WORKGROUP/MSTICAlertsWin1$Logon process: Advapi Authentication: NegotiateSource IpAddress: -Source Host: MSTICAlertsWin1Logon status: nan
Account: SYSTEMAccount Domain: NT AUTHORITYLogon Time: 2019-01-15 05:15:04.503Logon type: 5(Service)User Id/SID: S-1-5-18 SID S-1-5-18 is LOCAL_SYSTEMSubject (source) account: WORKGROUP/MSTICAlertsWin1$Logon process: Advapi Authentication: NegotiateSource IpAddress: -Source Host: -Logon status: nan
Account: adm1nistratorAccount Domain: MSTICAlertsWin1Logon Time: 2019-01-15 05:15:06.363Logon type: 3(Network)User Id/SID: S-1-5-21-996632719-2361334927-4038480536-1066 SID S-1-5-21-996632719-2361334927-4038480536-1066 is local machine or domain accountSubject (source) account: -/-Logon process: NtLmSsp Authentication: NTLMSource IpAddress: fe80::38dc:e4a9:61bd:b458Source Host: MSTICAlertsWin1Logon status: nan
Account: SYSTEMAccount Domain: NT AUTHORITYLogon Time: 2019-01-15 05:15:10.813Logon type: 5(Service)User Id/SID: S-1-5-18 SID S-1-5-18 is LOCAL_SYSTEMSubject (source) account: WORKGROUP/MSTICAlertsWin1$Logon process: Advapi Authentication: NegotiateSource IpAddress: -Source Host: -Logon status: nan
Account: SYSTEMAccount Domain: NT AUTHORITYLogon Time: 2019-01-15 05:15:14.453Logon type: 5(Service)User Id/SID: S-1-5-18 SID S-1-5-18 is LOCAL_SYSTEMSubject (source) account: WORKGROUP/MSTICAlertsWin1$Logon process: Advapi Authentication: NegotiateSource IpAddress: -Source Host: -Logon status: nan
